# 交叉验证及网格搜索

## 0. 加载相关模块

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 用于在jupyter中进行绘图
%matplotlib inline

## 1. 数据加载

In [2]:
iris = load_iris()

X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=0)

print('数据集样本数：{}，训练集样本数：{}，测试集样本数：{}'.format(len(X), len(X_train), len(X_test)))

数据集样本数：150，训练集样本数：112，测试集样本数：38


## 2. 交叉验证

交叉验证就是把训练集分成fold份，其中一份叫validation，另外fold-1份才是真正的训练集。对于超参的每个数值在所有的训练fold上跑一次并在validation上验证得到结果，取平均数就是该参数值对应的结果。选择结果最大的对应的超参。

然后给模型用选好的超参在测试集上测试。

KNN的超参：K,距离度量，。。。
SVM的超参： kernel，C，...

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score #返回对不同参数打分的一个列表

k_range = [1, 5, 9, 15]
cv_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)
    cv_score = np.mean(scores)
    print('k={}，验证集上的准确率={:.3f}'.format(k, cv_score))
    cv_scores.append(cv_score)
#CV还要自己拿最佳的超参重新训练一遍
best_k = k_range[np.argmax(cv_scores)]
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)
print('测试集准确率：', best_knn.score(X_test, y_test))  
    
    
    
    
#如果摆脱自己写循环,就是使用GridSearchCV构造单参数的线条：
from sklearn.model_selection import GridSearchCV
parameters = {'n_neighbors':k_range}
best_knn = GridSearchCV(KNeighborsClassifier(),parameters,cv=5,scoring='accuracy')
#这步喂给网格搜索数据使它能搜索到最优解，并且最后还会默认用最优解训练模型。但CV不会。
best_knn.fit(X_train,y_train) 
print('测试集准确率：', best_knn.score(X_test, y_test))
print(best_knn.best_params_)

k=1，验证集上的准确率=0.947
k=5，验证集上的准确率=0.955
k=9，验证集上的准确率=0.964
k=15，验证集上的准确率=0.964
测试集准确率： 0.9736842105263158
{'n_neighbors': 9}


/Users/apple/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## 3. 网格搜索

因为有多个超参需要候选，所以需要组合以后一个个挑选，就是grid_search.grid_search建立在cross-validation的基础上。

total running time: #hparam1 * #hparam2 * cv

In [19]:
from sklearn.model_selection import GridSearchCV #返回含有最佳超参数的模型实例
from sklearn.tree import DecisionTreeClassifier

parameters = {'max_depth':[3, 5, 7, 9], 'min_samples_leaf': [1, 2, 3, 4]}
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [20]:
print('最优参数：', clf.best_params_)
print('验证集最高得分：', clf.best_score_)

最优参数： {'max_depth': 5, 'min_samples_leaf': 2}
验证集最高得分： 0.9553571428571429


In [22]:
# 获取最优模型
best_model = clf.best_estimator_
print('测试集上准确率：', best_model.score(X_test, y_test))

测试集上准确率： 0.9473684210526315


## 4. 模型持久化

### 4.1 pickle

In [23]:
# 使用pickle
import pickle

model_path1 = './trained_model1.pkl'

# 保存模型到硬盘
with open(model_path1, 'wb') as f:
    pickle.dump(best_model, f)

In [25]:
# 加载保存的模型
with open(model_path1, 'rb') as f:
    model = pickle.load(f)

# 预测
print('预测值为', model.predict([X_test[0, :]]))
print('真实值为', y_test[0])

预测值为 [2]
真实值为 2


### 4.2 使用joblib

In [26]:
# 使用joblib
from sklearn.externals import joblib

# 保存模型到硬盘
model_path2 = './trained_model2.pkl'
joblib.dump(best_model, model_path2) 

['./trained_model2.pkl']

In [28]:
# 加载保存的模型
model = joblib.load(model_path2) 

# 预测
print('预测值为', model.predict([X_test[0, :]]))
print('真实值为', y_test[0])

预测值为 [2]
真实值为 2
